<a href="https://colab.research.google.com/github/Alwaz12/PIAIC-project-python.ipynb-colab/blob/main/day2b_fewshot_and_parsers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.6 MB/s eta 0:00:00


In [ ]:
!pip install -q -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm:ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    temprature = 0.2,
    model = "gemini-1.5-flash",
    api_key=GOOGLE_API_KEY
)

reposne = llm.invoke("What is the name of the Best Batsman India?")
to_markdown(reposne.content)

> There's no single definitive answer to the question of "the best batsman India" has ever produced.  It's a matter of opinion and depends on the criteria used (e.g., highest run scorer, most impactful player, best average, etc.).  Many greats are contenders, including:
> 
> * **Sachin Tendulkar:** Widely considered the greatest, holding numerous batting records.
> * **Virat Kohli:** Currently active, and a prolific run-scorer with an exceptional average.
> * **Sunil Gavaskar:** A legendary opener who dominated in his era.
> 
> Ultimately, choosing the "best" is subjective and depends on individual preferences.


# **1____________________FewShot Template____________________________________________**

# **Key Difference**
**FewShotPromptTemplate**: Gives examples to teach the AI how to perform the task.      
**ChatPromptTemplate**: Structures a chat or conversation with roles (System, Human, Assistant).

# **When to Use Each**
**FewShotPromptTemplate**: Use when the task is complex, and examples are needed to show how to do it.
**ChatPromptTemplate**: Use for chat-based tasks or when talking to the AI in a structured way.



**FewShotPromptTemplate**: Teach the AI with examples.                                                              
**ChatPromptTemplate**: Set up a chat with roles like System and Human.

# **Example 1**

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# Define examples with the correct keys matching the template
examples = [
    {"question": "What is 3 + 5?", "answer": "3 + 5 = 8."},
    {"question": "What is 7 * 6?", "answer": "7 * 6 = 42."}
]

# Define a template for the output format (this will be included in the prompt generation)
example_template = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question} \nAnswer: {answer}"
)

# Define the FewShotPromptTemplate using the examples
few_shot_template = FewShotPromptTemplate(
    example_prompt=example_template,  # Use the correct example_prompt parameter
    examples=examples,
    suffix="Question: {question}\nAnswer:",
)
# Format a new prompt with a new question
formatted_prompt = few_shot_template.format(question="What is 9 - 4?")

# Print the formatted prompt
print(formatted_prompt)


response = llm.invoke(formatted_prompt)

# Assuming the response contains the answer, convert it to markdown or display it
to_markdown(response.content)


Question: What is 3 + 5? 
Answer: 3 + 5 = 8.

Question: What is 7 * 6? 
Answer: 7 * 6 = 42.

Question: What is 9 - 4?
Answer:


> 9 - 4 = 5


# **Example 2**

incorrect syntex

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

examples = [
    {"question": "What is a dog?", "answer": "A dog is a domestic animal that is often kept as a pet."},
    {"question": "What is a cat?", "answer": "A cat is a small, domesticated animal known for its independence and playfulness."}
]

templates = PromptTemplate(
    template= "Question: {question}\nAnswer: {answer}",
    input_variables = ["question", "answer"]
)

few_shot_templates = FewShotPromptTemplate(
    example_prompt = templates,
    examples = examples,
    suffix = "Question /nAnswer:",
    input_variables = ["question"]
)

formated_prompt = few_shot_templates.format(question = "What is a dog?")

response = llm.invoke( formated_prompt)

to_markdown(response.content)


> Question: What is a bird?
> Answer: A bird is a warm-blooded vertebrate animal characterized by feathers, wings, and the laying of hard-shelled eggs.


# **Example 2 correct syntex**

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# Define examples with clear answers
examples = [
    {"question": "What is a dog?", "answer": "A dog is a domestic animal that is often kept as a pet."},
    {"question": "What is a cat?", "answer": "A cat is a small, domesticated animal known for its independence and playfulness."}
]

# Define a template for question-answer pairs
templates = PromptTemplate(
    template="Question: {question}\nAnswer: {answer}",
    input_variables=["question", "answer"]
)

# Define FewShotPromptTemplate
few_shot_templates = FewShotPromptTemplate(
    example_prompt=templates,  # Correct usage of example_prompt
    examples=examples,
    suffix="Question: {question}\nAnswer:",  # More clear instruction for the model
    input_variables=["question"]  # Specify question as the input variable
)

# Format the prompt with the new question
formated_prompt = few_shot_templates.format(question="What is a peregrine falcon?")

# Call the model
response = llm.invoke(formated_prompt)

# Display the response
to_markdown(response.content)


> A peregrine falcon is a large, powerful bird of prey known for its incredible speed in hunting.


# **Why this 1 Sentense is correct and another one is incorrect**

The second example is incorrect because the suffix uses "Question /nAnswer:" instead of the correct newline character "\n", causing the format to be incorrect and the model to misinterpret the structure.

# **Parsers in Langchain**

**DateTimeOutPutParser**

**1st**

In [ ]:

from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

date_Time_Parser = DatetimeOutputParser()

human_template = "{request} /n {format_instructions}"

chat_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(human_template)
])

formatted_messages = chat_prompt.format_messages(
    request="What was the date When india became independent",
    format_instructions=date_Time_Parser.get_format_instructions()
)

llm_response=llm.invoke(formatted_messages)

print(llm_response.content)
print("date Time Parser", date_Time_Parser.parse(llm_response.content))

1947-08-15T00:00:00.000000Z

date Time Parser 1947-08-15 00:00:00


**2nd**

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import DatetimeOutputParser

human_template = "{user_request} /n {llm_instruction}"

prompt_chat = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(human_template)
])

formatted_messages = prompt_chat.format_messages(
    user_request="tell me something about pakistan indepedence day",
    llm_instruction="Provide a brief summary"
)

llm_reponse = llm.invoke(formatted_messages)
to_markdown(llm_reponse.content)


> Pakistan's Independence Day, celebrated annually on August 14th, commemorates the nation's independence from British rule in 1947.  The day marks the end of the British Raj and the creation of Pakistan as a separate, independent Muslim-majority state.  The event was part of the Partition of India, a complex and often violent process that resulted in the division of British India into India and Pakistan.  Independence Day is a national holiday featuring parades, flag hoisting ceremonies, and celebratory events across the country.


# **Pyndatic OutPut Parser**

**1st**

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import DatetimeOutputParser,PydanticOutputParser
from pydantic import BaseModel,Field

class Cricketer(BaseModel):
  name: str = Field(description="Name os the cricketer")
  records: str = Field(description="Records of the cricketer")

parser = PydanticOutputParser(pydantic_object=Cricketer)

human_template = "{user_request} /n {user_instruction}"

prompt_chat = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(human_template)
])

formatted_chat_prompt = prompt_chat.format_messages(
    user_request="Tell me about Cricketer",
    user_instruction=parser.get_format_instructions()
)

llm_response = llm.invoke(formatted_chat_prompt)
to_markdown(llm_response.content)


> ```json
> {
>   "name": "Sachin Tendulkar",
>   "records": "Considered one of the greatest batsmen of all time, Sachin Tendulkar holds numerous records in cricket, including the most runs scored in both Test and ODI cricket.  He also holds the record for the most centuries scored in both formats."
> }
> ```


**2nd** in List Formate

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import DatetimeOutputParser,PydanticOutputParser
from pydantic import BaseModel,Field

class Cricketer(BaseModel):
  name: str = Field(description="Name of the cricketer")
  records: str = Field(description="Records of the cricketer")

parser = PydanticOutputParser(pydantic_object=Cricketer)

human_template = "{user_request} /n Please provide the response in list formate about Sarfaraz Ahmed."

prompt_chat = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(human_template)
])

formatted_chat_prompt = prompt_chat.format_messages(
    user_request="Tell me about Cricketer",
    user_instruction=parser.get_format_instructions()
)

llm_response = llm.invoke(formatted_chat_prompt)
# print(llm_response.content)
to_markdown(llm_response.content)


> Sarfaraz Ahmed:
> 
> * **Full Name:** Sarfaraz Ahmed
> * **Born:** 25 December 1987 (age 35)
> * **Nationality:** Pakistani
> * **Major Teams:**  Pakistan national cricket team, Sindh, Quetta Gladiators, Karachi Kings
> * **Batting Style:** Right-handed
> * **Bowling Style:** Right-arm leg break
> * **Role:** Wicket-keeper batsman
> * **International Debut:**
>     * Test: 2010
>     * ODI: 2007
>     * T20I: 2007
> * **Captaincy:**  Captained the Pakistan national cricket team in all three formats.  Noteworthy period as captain was from 2017-2019.
> * **Strengths:**  Excellent wicket-keeper, capable middle-order batsman, strong leadership qualities (although his captaincy faced criticism).
> * **Achievements:** Led Pakistan to Champions Trophy victory in 2017.
> * **Known for:**  His aggressive batting style, sharp wicket-keeping, and (controversially at times) his captaincy.
> * **Current Status:**  Continues to play domestic and franchise cricket.  His international career has seen fluctuations in selection.


**3rd** in JSON Formate

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import DatetimeOutputParser,PydanticOutputParser
from pydantic import BaseModel,Field

class Cricketer(BaseModel):
  name: str = Field(description="Name of the cricketer")
  records: str = Field(description="Records of the cricketer")

parser = PydanticOutputParser(pydantic_object=Cricketer)

human_template = "{user_request} /n Please provide the response in JSON formate about Sarfaraz Ahmed."

prompt_chat = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(human_template)
])

formatted_chat_prompt = prompt_chat.format_messages(
    user_request="Tell me about Cricketer",
    user_instruction=parser.get_format_instructions()
)

llm_response = llm.invoke(formatted_chat_prompt)
to_markdown(llm_response.content)


> ```json
> {
>   "name": "Sarfaraz Ahmed",
>   "nationality": "Pakistani",
>   "role": "Wicket-keeper batsman",
>   "born": "December 22, 1987",
>   "battingStyle": "Right-hand bat",
>   "bowlingStyle": "Right-arm leg break",
>   "teams": [
>     "Sindh",
>     "Quetta Gladiators",
>     "Pakistan"
>   ],
>   "careerHighlights": [
>     {
>       "title": "Captaincy",
>       "details": "Successfully led Pakistan in various formats, notably the 2017 Champions Trophy victory."
>     },
>     {
>       "title": "Wicket-keeping Prowess",
>       "details": "Known for his sharp and agile wicket-keeping skills, often making difficult catches and stumpings."
>     },
>     {
>       "title": "Aggressive Batsmanship",
>       "details": "A capable middle-order batsman known for his aggressive and attacking style of play."
>     },
>     {
>       "title": "ODI Career",
>       "details": "Significant contributions in ODI cricket, including crucial innings and match-winning performances."
>     },
>     {
>       "title": "Test Career",
>       "details": "Has played test matches for Pakistan, but with varying degrees of success."
>     },
>     {
>       "title": "T20 Career",
>       "details": "A key player in Pakistan's T20 side, known for his explosive batting in the shorter format."
>     }
>   ],
>   "significantAchievements": [
>     "2017 Champions Trophy Winner (Captain)",
>     "Numerous match-winning performances across various formats"
>   ],
>   "knownFor": [
>     "Leadership",
>     "Wicket-keeping",
>     "Aggressive batting",
>     "Match-winning ability"
>   ],
>   "controversies":[
>     "Experienced periods of fluctuating form and faced criticism at different points in his career."
>   ]
> }
> ```


## **Comma Seprated List Output Parser**

**1st**

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

human_templates = "{user_request} /n {formate_instructons}"

chat_prompts = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(human_templates)
])

formatted_chats = chat_prompts.format_messages(
    user_request="Tell me about Best Picnic Places in paksitan",
    formate_instructons=parser.get_format_instructions()
)

llm_response = llm.invoke(formatted_chats)
to_markdown(llm_response.content)


> Frere Hill Gardens,Karachi,Lawrence Gardens,Lahore,Jinnah Garden,Rawalpindi,Shakarparian Park,Islamabad,Lake View Park,Faisalabad,Astola Island,Bela,Kund Malir Beach,Gwadar,Neela Sand Beach,Ormara,Ranikot Fort,Thatta
